In [1]:
# 코랩에 구글 드라이브 마운트
from google.colab import drive
drive.mount("/content/drive")
import warnings
# warnings.filterwarnings(action = "ignore")

# 데이터 처리 모듈
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import time
from tqdm import tqdm

# 멀티프로세싱
import multiprocessing as mp
from multiprocessing import Pool

# 척도 변환 모듈(표준화, 정규화, 로버스트 정규화, 원핫인코딩(범주형에서 더미변수로 변환) 등)
from sklearn.preprocessing import *

# 결측값 관측 모듈
import missingno as msno

# 시각화 모듈
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from matplotlib.ticker import ScalarFormatter

# 배열, 행렬 연산 모듈
import numpy as np

# 데이터 분할 모듈(훈련용 / 검증용 / 시험용)
from sklearn.model_selection import train_test_split

# 통계적 가설검정 및 계량화 모듈
# https://youtu.be/FtWEZw3kUho
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from scipy.interpolate import UnivariateSpline

# 타입 어노테이션 모듈
from typing import *

# 시간 관련 모듈
from tqdm import tqdm
from time import strptime, sleep

# 웹크롤링 / 스크래핑 관련 모듈
import requests
import io
import zipfile
from bs4 import BeautifulSoup
from xml.etree import ElementTree as ET

# 결측값 대체 및 특징 추출
from sklearn.impute import KNNImputer
from sklearn.decomposition import PCA, SparsePCA

# 연관규칙 분석 실행 모듈
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
import networkx as nx

# 한글폰트 설정
import matplotlib
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from pathlib import Path
import matplotlib.patches as mpatches

Mounted at /content/drive


In [ ]:
# 주식종목 데이터 로드 모듈
!pip install pykrx
from pykrx import stock
!pip install finance-datareader --upgrade
import FinanceDataReader as fdr

In [ ]:
class IndexDatawithEventMoment():

  def __init__(self, file_path: str, DROP_COLUMN_LIST: List, SUBSET = None):

    self.INDEX_NEWS_DATA = pd.read_csv(file_path, index_col = False, na_values = "NaN")
      # [file_path] "/content/drive/MyDrive/AfterLearnerProject/DataArchive/merge_derivative_news_data.csv"
      # [INDEX_PRINCIPAL_COMPONENT_x] 시계열 전처리한 5개 주가지수선형결합 제1주성분
      # [INDEX_PRINCIPAL_COMPONENT_y] 시계열 전처리한 5개 주가지수선형결합 제1주성분의 단위변동률
    self.INDEX_NEWS_DROPNA = self.INDEX_NEWS_DATA.dropna(subset = DROP_COLUMN_LIST)
      # msno.dendrogram()
    self.SUBSET = SUBSET


  def compareEventDatetimewithNewsImportanceScore(self):
    # improtance와 특이 지점을 비교하기 위해 importance값 상위 20개 추출(특이지점과 흡사)
    display( self.INDEX_NEWS_DATA.sort_values(
        by = "IMPORTANCE", ascending = False).head(20)[["IMPORTANCE", "YYYYMMDD", "TAG_LIST"]] )


  def removeBlank(self):
    import re
    self.INDEX_NEWS_DROPNA["TAG_LIST"] = self.INDEX_NEWS_DROPNA["TAG_LIST"].apply(
        lambda tag: re.sub(" ", "", tag))
    self.INDEX_NEWS_DROPNA["ITEM_NAME"] = self.INDEX_NEWS_DROPNA["ITEM_NAME"].apply(lambda tag: re.sub(" ", "", tag))
      # "\s{0,}"
      # re.sub(" +", "" , string)
      # " ".join(string.split())


  def mutateNewColumn(self):
    self.INDEX_NEWS_DROPNA["CHANGE_CATEGORY"] = (
        self.INDEX_NEWS_DROPNA["STABLE_or_DRAMATIC"].astype(str) + "_" + self.INDEX_NEWS_DROPNA["SIGN_CHANGE"].astype(str))
    self.INDEX_NEWS_DROPNA["TAG_SPLIT"] = self.INDEX_NEWS_DROPNA.TAG_LIST.str.split("|")
    self.INDEX_NEWS_DROPNA["ITEM_SPLIT"] = self.INDEX_NEWS_DROPNA.ITEM_NAME.str.split("|")
    print( "\n", Counter(self.INDEX_NEWS_DROPNA["CHANGE_CATEGORY"]) , "\n", Counter(self.INDEX_NEWS_DROPNA["YYYYMMDD"]) )
    return self.INDEX_NEWS_DROPNA


  def makeSubDataset_loadData(self, date_column: str, date_time: str):

    INDEX_NEWS_DATETIME = self.INDEX_NEWS_DROPNA[ self.INDEX_NEWS_DROPNA[date_column] == date_time ]
    self.SUBSET = INDEX_NEWS_DATETIME[["CHANGE_CATEGORY", "SUBCATEGORY"]].values
      # [column] "ITEM_NAME", "SUBCATEGORY"
    return self.SUBSET, INDEX_NEWS_DATETIME
      # [date_column] YYYYMMDD

In [ ]:
MarketIndexObject = IndexDatawithEventMoment(
    file_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/merge_derivative_news_data.csv",
    DROP_COLUMN_LIST = ["TAG_LIST", "ITEM_NAME"]
)

MarketIndexObject.compareEventDatetimewithNewsImportanceScore()
MarketIndexObject.removeBlank()
INDEX_NEWS_DATA = MarketIndexObject.mutateNewColumn()
# Counter({'2023-03-15': 2918, '2023-03-10': 2152, '2023-04-12': 1858, '2023-02-01': 1687, '2023-01-31': 1621, '2023-02-02': 1565, '2023-01-26': 1376, '2023-01-25': 1294, '2023-01-27': 1062})

**2023-01-25** | DRAMATIC_Not Change
(AI)	(인공지능) | (인공지능)	(AI)

(은행)	(금리) | (대출)	(금리)

**2023-01-26** | DRAMATIC_Not Change

(대출)	(금리) | (금리, 은행)	(대출)

(무인기)	(북한)	| (북한)	(무인기)

**2023-01-27** | DRAMATIC_Not Change
(마스크)	(코로나19) | (코로나19) (마스크)

(전기차)	(중국)	| (전기차)	(글로벌)

(전기차)  (배터리)	| (배터리)	(전기차)

**2023-01-31** | DRAMATIC_Not Change
(반도체)	(메모리) | (메모리)	(반도체) | (파운드리)	(반도체)

(반도체)	(글로벌) | (일본)	(중국) | (코로나19)	(중국) | (소비)	(중국)

(부동산)	(주택)	| (주택)	(부동산)

**2023-02-01** | DRAMATIC_Sign Change

(중국)	(글로벌) | (리오프닝)	(중국)

(대출)	(금리)

(반도체)	(메모리)

**2023-02-02** | DRAMATIC_Sign Change

(스마트폰)	(갤럭시) | (갤럭시)	(스마트폰) | (카메라)	(갤럭시) | (디지털)	(플랫폼)

(금리)	(중앙은행) | (금융)  (금리) | (채권)	(금리)

(배터리)	(전기차)	|  (전기차)	(배터리) | (전기)	(가스)

**2023-03-10** | DRAMATIC_Not Change

(이머징마켓, 아시아)	(미국) | (미국, 이머징마켓)	(아시아) | (중국, 아시아)	(이머징마켓)

(EU, 미국) | (, 소재)	(미국)

(채권)	(금리)

**2023-04-12** | DRAMATIC_Not Change
(미국, 아시아)	(이머징마켓) | (미국, 이머징마켓)	(아시아) | (아시아, 이머징마켓)	(미국)
(소재)	(미국) | (헬스케어)	(미국)	| (미세먼지)	(황사)
(중국, 이머징마켓)	(아시아) | (EU, 아시아)	(이머징마켓)


## **CS_DATA**와 MARKET_INDEX_NEWS_DATA 8가지 시점별 병합

* 시계열 정보 : 4개월간의 자산 및 거래, 접속내역 데이터
* 후위 표시자로 시기 구분
* 자산내역 : 월말일 기준 데이터 제공

  M1(2022.12), M2(2023.01), M3(2023.02), M4(2023.03)
* 거래내역, 접속내역 : 월별 초순, 중순, 하순 동안의 데이터 집계내역 제공

  M2_1(2023.01 초순 1 ~ 10일), M2_2(2023.01 중순 11 ~ 20일), M2_3(2023.01 하순 21 ~ 말일)을 _1, _2, _3으로 구분
  
* 『전략적 인출 설계와 은퇴 포트폴리오의 과학』


#### **1.필요한 데이터 처리하는 함수 및 전처리된 데이터**

In [ ]:
class GetStockCodeData():

  def __init__(self,
               KOR_SECTOR = None):
    self.KOR_SECTOR = KOR_SECTOR


  def getStockCode(self, date, market):

    stock_list = pd.DataFrame(
        {"종목코드": stock.get_market_ticker_list(date, market = market)})

    stock_list["종목명"] = stock_list["종목코드"].map(lambda x: stock.get_market_ticker_name(x))
    # https://github.com/financedata-org/FinanceDataReader/blob/develop/krx/data.py
    # fdr_data = fdr.StockListing("KRX")
    # stock_list["업종"] = stock_list["종목명"].map(lambda x: fdr_data[fdr_data["Name"] == x]["Sector"].iloc[0])
    return stock_list


  def getStockList(self, base_date):

    self.KOSPI_TICKER = self.getStockCode(date = base_date, market = "KOSPI")
    self.KOSDAQ_TICKER = self.getStockCode(date = base_date, market = "KOSDAQ")


  def getStockSector(self, sector_code_list: List):

    DATA_SECTOR: List = []
    for sector_code in sector_code_list:
        URL = f'''https://www.wiseindex.com/Index/GetIndexComponets?ceil_yn=0&dt=20230712&sec_cd={ sector_code }'''
        DATA = requests.get(URL).json()
        DATA_JSON = pd.json_normalize(DATA["list"])
        DATA_SECTOR.append(DATA_JSON)
        time.sleep(2)
    KOR_SECTOR = pd.concat(DATA_SECTOR, axis = 0)
    KOR_SECTOR.rename(columns = {"CMP_CD": "TICKER"}, inplace = True)
    self.KOR_SECTOR = KOR_SECTOR[["TICKER", "SEC_NM_KOR"]]

    # sector_code_list = ["G10","G15","G20","G25", "G30", "G35", "G40", "G45","G50", "G55"]


  def getFundamental(self, base_date: str):

    KOSPI_FUNDAMENTAL = stock.get_market_fundamental_by_ticker(
        date = base_date, market = "KOSPI")
    KOSPI_FUNDAMENTAL["종목코드"] = KOSPI_FUNDAMENTAL.index
    KOSPI_FUNDAMENTAL.reset_index(drop = True, inplace = True)
    self.KOSPI_TICKER = pd.merge(
        self.KOSPI_TICKER, KOSPI_FUNDAMENTAL,
        left_on = "종목코드", right_on = "종목코드", how = "outer")
    self.KOSPI_TICKER = pd.merge(
        self.KOSPI_TICKER, self.KOR_SECTOR,
        left_on = "종목코드", right_on = "TICKER", how = "left")

    KOSDAQ_FUNDAMENTAL = stock.get_market_fundamental_by_ticker(
        date = base_date, market = "KOSDAQ")
    KOSDAQ_FUNDAMENTAL["종목코드"] = KOSDAQ_FUNDAMENTAL.index
    KOSDAQ_FUNDAMENTAL.reset_index(drop = True, inplace = True)
    self.KOSDAQ_TICKER = pd.merge(
        self.KOSDAQ_TICKER, KOSDAQ_FUNDAMENTAL,
        left_on = "종목코드", right_on = "종목코드", how = "outer")
    self.KOSDAQ_TICKER = pd.merge(
        self.KOSDAQ_TICKER, self.KOR_SECTOR,
        left_on = "종목코드", right_on = "TICKER", how = "left")

    return self.KOSPI_TICKER, self.KOSDAQ_TICKER


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
Indicator = GetStockCodeData()
# STOCK_CODE_LIST = Indicator.getStockCode()
Indicator.getStockList(base_date = "2023-03-31")
Indicator.getStockSector(sector_code_list = ["G10","G15","G20","G25", "G30", "G35", "G40", "G45","G50", "G55"])
KOSPI_DATA, KOSDAQ_DATA = Indicator.getFundamental(base_date = "2023-03-31")

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
STOCK_LIST = pd.concat(
    [ KOSPI_DATA, KOSDAQ_DATA ],
    axis = 0, ignore_index = True)

* 특정 기간 동안의 주식 매수액 1(2, 3)위 종목번호

```
CS_BUY_M2_3 = pd.DataFrame()
CS_BUY_M3_1 = pd.DataFrame()
CS_BUY_M4_2 = pd.DataFrame()

for i in range(len(consumer_data)):
  CS_BUY_M2_3 = pd.concat(
      [ CS_BUY_M2_3, consumer_data[i].loc[:, ["BUY1_ITM_M2_3", "BUY2_ITM_M2_3", "BUY3_ITM_M2_3"]] ]).dropna()
  CS_BUY_M3_1 = pd.concat(
      [ CS_BUY_M3_1, consumer_data[i].loc[:, ["BUY1_ITM_M3_1", "BUY2_ITM_M3_1", "BUY3_ITM_M3_1"]] ]).dropna()
  CS_BUY_M4_2 = pd.concat(
      [ CS_BUY_M4_2, consumer_data[i].loc[:, ["BUY1_ITM_M4_2", "BUY2_ITM_M4_2", "BUY3_ITM_M4_2"]] ]).dropna()

CS_BUY_M2_3["CONSUMER_ID"] = CS_BUY_M2_3.index
CS_BUY_M3_1["CONSUMER_ID"] = CS_BUY_M3_1.index
CS_BUY_M4_2["CONSUMER_ID"] = CS_BUY_M4_2.index
```

* 특정 기간 동안의 주식 매도액 1(2, 3)위

```
CS_SELL_M2_3 = pd.DataFrame()
CS_SELL_M3_1 = pd.DataFrame()
CS_SELL_M4_2 = pd.DataFrame()

for i in range(len(consumer_data)):
  CS_SELL_M2_3 = pd.concat(
      [ CS_SELL_M2_3, consumer_data[i].loc[:, ["SEL1_ITM_M2_3", "SEL2_ITM_M2_3", "SEL3_ITM_M2_3"]] ]).dropna()
  CS_SELL_M3_1 = pd.concat(
      [ CS_SELL_M3_1, consumer_data[i].loc[:, ["SEL1_ITM_M3_1", "SEL2_ITM_M3_1", "SEL3_ITM_M3_1"]] ]).dropna()
  CS_SELL_M4_2 = pd.concat(
      [ CS_SELL_M4_2, consumer_data[i].loc[:, ["SEL1_ITM_M4_2", "SEL2_ITM_M4_2", "SEL3_ITM_M4_2"]] ]).dropna()
# Counter(CS_SELL_M2_3["SEL1_ITM_M2_3"])

CS_SELL_M2_3["CONSUMER_ID"] = CS_SELL_M2_3.index
CS_SELL_M3_1["CONSUMER_ID"] = CS_SELL_M3_1.index
CS_SELL_M4_2["CONSUMER_ID"] = CS_SELL_M4_2.index
```

___

#### **2.동일한 시기에 투자심리에 영향을 받는 투자자들은 체계적인 투자패턴을 보여야 한다는 가정**

1) 특정 시점의 투자심리지수 외 행동패턴 데이터가 존재하는 고객 추출

2) 클러스터 생성

3) 클러스터별 고객 표본집단을 가지고 투자패턴 / 행태에 통계적으로 유의적인 차이가 있는지 가설검정

In [ ]:
class LoadData():

# 클래스(로컬공간) 내부에서 cs_data, apy_itm_hist_dat, market_index_data의 값
# 생성자를 활용하여 초기화하고 클래스 내부에 캡슐화
  def __init__(self,
               cs_data_path: str, apy_itm_hist_path: str):
    # 용량이 큰 csv 파일 읽어오기(fopen - fread와 유사한 방식)
    cs_chunk = pd.read_csv(cs_data_path, chunksize = 10**5, index_col = False, na_values = "NaN", encoding = "utf-8")
    self.cs_data = list(cs_chunk)
    self.apy_itm_hist_data = pd.read_csv(apy_itm_hist_path, index_col = False, na_values = "NaN", encoding = "utf-8")
      # [BAS_YM] (19.12 ~ 23.04) 기준연월
      # [APY_ITM] (19.12 ~ 23.04)간 공모주 청약 진행된 종목 번호
      # [APY_FIN_YM] 이전 3년(19.12 ~ 22.11)간 마지막으로 참여한 공모주 청약연월(X)
      # [APY_ITM_CNT] 이전 3년(19.12 ~ 22.11)간 공모주 청약에 참여한 종목 개수(X)


# parallelDataFrame()
# csv(쉼표로 분할된) 데이터 로드 시 병렬처리하는 메서드
# 실제로 수행할 정도로 데이터 크기가 크지는 않고 100,000행씩 chunking하면 해결됨
  def parallelDataFrame(self, data, function):
    # 코어 수 확인(multiprocessing as mp)
    mp.cpu_count()
    # 코어 수를 전역변수 선언
    global num_cores
    # 코어 수만큼 입력받은 데이터를 2차원 배열로 분할하여 멀티프로세싱(병렬처리)
    data_split = np.array_split(data, num_cores)
    # 코어 수만큼 pool을 생성
    pool = mp.Pool(num_cores)
    # 나누어진 데이터를 function으로 넘겨서 병렬처리 수행
    data = pd.concat(pool.map(
        function, iterable = data_split))
    pool.close()
    # 모두 완료될 때까지 대기
    pool.join
    return data


# loadData()
# 불러온 데이터를 반환(코드 구현 시 활용하고 나중에는 제거할 메서드)
  def loadData(self):
    return self.cs_data, self.apy_itm_hist_data

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
DataInstance = LoadData(
    cs_data_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/cs_data.csv",
    apy_itm_hist_path = "/content/drive/MyDrive/AfterLearnerProject/DataArchive/apy_itm_hist.csv",
)
  # [참고] 경로 너무 길어지니까 겹치는 경로는 os.path.join 활용하여 반복 줄일 수 있음

In [ ]:
consumer_data, apply_item_histogram = DataInstance.loadData()
  # cs_data, apy_item_hist 데이터

# consumer_merge_data = pd.concat(consumer_data)
  # [참고] 텍스트 파일로 저장해도 1.23GB 용량은 줄지 않는바 수행하지 않음
  # cs_merge_data.to_csv("/content/drive/MyDrive/AfterLearnerProject/DataArchive/cs_data.txt", index = False)

# _, column_length = consumer_merge_data.shape

# 총 552개 컬럼(dtypes: float64(325), int64(2), object(195))으로
# 컬럼별 타입 및 결측값 확인 반복문 수행 필요

# for index in range(0, column_length, 50):
   # consumer_merge_data.iloc[:, index : index + 50].info()

# msno.bar(consumer_merge_data)
  # [참고] 데이터 상세상 0과 결측(NULL)의 구분이 없다는 점에서
  # 함부로 결측값을 예측모델로 대체하면 안됨

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [23]:
def SelectCounsumerDataColumn(column_list_M2_3: List,
                              column_list_M3_1: List,
                              column_list_M4_2: List,
                              column_name_list: List):

  CS_DATA_M2_3 = pd.DataFrame()
  CS_DATA_M3_1 = pd.DataFrame()
  CS_DATA_M4_2 = pd.DataFrame()

  for i in range(len(consumer_data)):
    CS_DATA_M2_3 = pd.concat(
        [ CS_DATA_M2_3, consumer_data[i].loc[:, column_list_M2_3] ]
    )
    # .fillna(0)
    # .dropna()
    CS_DATA_M3_1 = pd.concat(
        [ CS_DATA_M3_1, consumer_data[i].loc[:, column_list_M3_1] ]
    )
    # .fillna(0)
    # .dropna()
    CS_DATA_M4_2 = pd.concat(
        [ CS_DATA_M4_2, consumer_data[i].loc[:, column_list_M4_2] ]
    )
    # .fillna(0)
    # .dropna()

  CS_DATA_M2_3["CONSUMER_ID"] = CS_DATA_M2_3.index
  CS_DATA_M3_1["CONSUMER_ID"] = CS_DATA_M3_1.index
  CS_DATA_M4_2["CONSUMER_ID"] = CS_DATA_M4_2.index

  CS_DATA_M2_3.columns = column_name_list
  CS_DATA_M3_1.columns = column_name_list
  CS_DATA_M4_2.columns = column_name_list

  return CS_DATA_M2_3, CS_DATA_M3_1, CS_DATA_M4_2

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [24]:
CS_DATA_M2_3, CS_DATA_M3_1, CS_DATA_M4_2 = SelectCounsumerDataColumn(
    column_list_M2_3 = [
        "CASH_AST_M2", "DMST_AST_EVAL_M2", "DMST_AST_PCHS_M2", "OVST_AST_EVAL_M2", "OVST_AST_PCHS_M2", "DMST_ITM_CNT_M2", "OVST_ITM_CNT_M2",
        "BUY_CNT_M2_3", "SEL_CNT_M2_3", "BUY_ITM_CNT_M2_3", "SEL_ITM_CNT_M2_3", "BUY_AMT_M2_3", "SEL_AMT_M2_3",
        "STK_IN_M2_3", "STK_OUT_M2_3", "MNY_IN_M2_3", "MNY_OUT_M2_3", "CONN_DYS_M2_3", "MTS_DYS_M2_3", "HTS_DYS_M2_3",
        "AGE_TCD", "MT_EP_EXIST_YN", "LST_BEST_EA", "LST_BEST_YM"],

    column_list_M3_1 = [
        "CASH_AST_M3", "DMST_AST_EVAL_M3", "DMST_AST_PCHS_M3", "OVST_AST_EVAL_M3", "OVST_AST_PCHS_M3", "DMST_ITM_CNT_M3", "OVST_ITM_CNT_M3",
        "BUY_CNT_M3_1", "SEL_CNT_M3_1", "BUY_ITM_CNT_M3_1", "SEL_ITM_CNT_M3_1", "BUY_AMT_M3_1", "SEL_AMT_M3_1",
        "STK_IN_M3_1", "STK_OUT_M3_1", "MNY_IN_M3_1", "MNY_OUT_M3_1", "CONN_DYS_M3_1", "MTS_DYS_M3_1", "HTS_DYS_M3_1",
        "AGE_TCD", "MT_EP_EXIST_YN", "LST_BEST_EA", "LST_BEST_YM"],

    column_list_M4_2 = [
        "CASH_AST_M4", "DMST_AST_EVAL_M4", "DMST_AST_PCHS_M4", "OVST_AST_EVAL_M4", "OVST_AST_PCHS_M4", "DMST_ITM_CNT_M4", "OVST_ITM_CNT_M4",
        "BUY_CNT_M4_2", "SEL_CNT_M4_2", "BUY_ITM_CNT_M4_2", "SEL_ITM_CNT_M4_2", "BUY_AMT_M4_2", "SEL_AMT_M4_2",
        "STK_IN_M4_2", "STK_OUT_M4_2", "MNY_IN_M4_2", "MNY_OUT_M4_2", "CONN_DYS_M4_2", "MTS_DYS_M4_2", "HTS_DYS_M4_2",
        "AGE_TCD", "MT_EP_EXIST_YN", "LST_BEST_EA", "LST_BEST_YM"],

    column_name_list = [
        "월말일 기준 예수금 등 현금성 자산액", "월말일 기준 국내주식 자산 평가액", "월말일 기준 국내주식 자산 매입액", "월말일 기준 해외주식 자산 평가액", "월말일 기준 해외주식 자산 매입액",
        "월말일 기준 국내주식 보유종목수", "월말일 기준 해외주식 보유종목수", "특정기간동안의 주식 매수체결횟수", "특정기간동안의 주식 매도체결횟수", "특정기간동안의 주식 매수종목수", "특정기간동안의 주식 매도종목수",
        "특정기간동안의 주식 매수액 합계", "특정기간동안의 주식 매도액 합계", "특정기간동안의 주식 입고액 합계", "특정기간동안의 주식 출고액 합계", "특정기간동안의 입금액 합계", "특정기간동안의 출금액 합계",
        "특정기간동안의 HTS 또는 MTS 접속일수", "특정기간동안의 MTS 접속일수", "특정기간동안의 HTS 접속일수",
        "10세 기준 연령대", "관리사원 존재여부", "과거 최고자산액", "과거 최고자산액 기록 연월", "고객번호"]
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


**6개의 투자심리 측정변수**
* NYSE의 거래회전율
* 배당금 프리미엄
* 폐쇄형 펀드의 할인
율
* IPO의 첫 거래일 평균 수익률
* 채권 대비 주식발행 비중
* 투자심리선은 해당 종목이 과매수 또는 과매도 상태인지를 파악함으로써 투자자 심리가 낙관적인지 비관적인지를 판단하는데 사용하며, 일반적으로 75% 이상이면 해당 주식이 과매수 상태, 25% 이하이면 과매도 상태로 해석

**6개 변수를 이용하여 월별 투자 심리지수를 도출**
* ```개인투자자의 매수매도 불균형(BSI, buying and selling imbalance of individual investors)```

* ```주식형 펀드 유입액(FUND, stock fund flow ratio)```

  예금이나 채권형 펀드에서 위험한 주식형 펀드로 자금이 이동하는 현상

* ```소비자 기대지수(CEI, customer expectation index)```
  
  통계청에서 발간하는 소비자 기대지수는 경기 순환을 예측하는 지수로 본 연구에서는 자연로그를 취하여 사용(경기선행지수 중 1)

  (i) 현재생활형편, 생활형편전망, 가계수입전망, 소비지출전망, 현재경기판단, 향후경기전망 6개의 개별지수의 합산지수 산출

  (ii) 표준화하고, 6개 구성지수 표준편차의 중간값을 가중치로 곱하여 소비자 심리지수 산출

  (iii) 100보다 높으면 주관적 기대심리 과거 평균 대비 낙관적으로 해석(기준치 상회 여부 및 지수 자체의 증감 중요)

* ```주식투자 예치금(CD, customer’s deposit for stock investment)```
  
  주식투자를 위해 예치한 금액이 얼마나 변동하였는지를 측정하여 추가적으로 고려한다. 주식투자 예치금은 주식을 매수하기 위한 일시적인 예치금으로 투자자들이 주식시장에 대해 보다 낙관적인 견해를 가지고 있을수록 증가

* ```상장주식 거래회전율율(TURN, turnover ratio)```

* ```주식 자금조달 비율(SR, equity shares issued ratio)```

  주식과 채권을 합계한 총 자금 조달금액에서 주식발행을 통해 조달한 자금의 비중을 사용

In [28]:
pd.set_option("display.max_column", None)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
for dataframe in [CS_DATA_M2_3, CS_DATA_M3_1, CS_DATA_M4_2]:
                    # 150446 | # 194670 | # 168290
  dataframe.iloc[:, 7:13].info()
  dataframe.dropna(axis = 0, subset = [
      "특정기간동안의 주식 매수체결횟수",
      "특정기간동안의 주식 매도체결횟수",
      "특정기간동안의 주식 매수종목수",
      "특정기간동안의 주식 매도종목수",
      "특정기간동안의 주식 매수액 합계",
      "특정기간동안의 주식 매도액 합계"], inplace = True)
  display(dataframe)
    # print(Counter(dataframe["관리사원 존재여부"]))
    # Counter({'N': 712145, 'Y': 91163})

    # 특정기간동안의 주식 매수체결횟수/매수종목수/매수액합계의 특이시점별 동일 고객에게 결측값 존재
    # 특정기간동안의 주식 매도체결횟수/매도종목수/매도액합계 특이시점별 동일 고객에게 결측값 존재

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803308 entries, 0 to 803307
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   특정기간동안의 주식 매수체결횟수  216743 non-null  float64
 1   특정기간동안의 주식 매도체결횟수  202784 non-null  float64
 2   특정기간동안의 주식 매수종목수   216743 non-null  float64
 3   특정기간동안의 주식 매도종목수   202784 non-null  float64
 4   특정기간동안의 주식 매수액 합계  216743 non-null  float64
 5   특정기간동안의 주식 매도액 합계  202784 non-null  float64
dtypes: float64(6)
memory usage: 36.8 MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,월말일 기준 예수금 등 현금성 자산액,월말일 기준 국내주식 자산 평가액,월말일 기준 국내주식 자산 매입액,월말일 기준 해외주식 자산 평가액,월말일 기준 해외주식 자산 매입액,월말일 기준 국내주식 보유종목수,월말일 기준 해외주식 보유종목수,특정기간동안의 주식 매수체결횟수,특정기간동안의 주식 매도체결횟수,특정기간동안의 주식 매수종목수,특정기간동안의 주식 매도종목수,특정기간동안의 주식 매수액 합계,특정기간동안의 주식 매도액 합계,특정기간동안의 주식 입고액 합계,특정기간동안의 주식 출고액 합계,특정기간동안의 입금액 합계,특정기간동안의 출금액 합계,특정기간동안의 HTS 또는 MTS 접속일수,특정기간동안의 MTS 접속일수,특정기간동안의 HTS 접속일수,10세 기준 연령대,관리사원 존재여부,과거 최고자산액,과거 최고자산액 기록 연월,고객번호
2,350.0,31000000.0,51000000.0,5600000.0,10000000.0,26.0,6.0,20.0,17.0,17.0,17.0,50000000.0,51000000.0,NaN,NaN,NaN,NaN,11.0,11.0,NaN,40,N,4.400000e+07,202206,2
3,49000000.0,NaN,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,2.0,100000000.0,150000000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,40,Y,2.100000e+08,202201,3
5,350000.0,83000000.0,150000000.0,NaN,NaN,2.0,NaN,4.0,2.0,1.0,1.0,4700000.0,3700000.0,NaN,NaN,1800000.0,500000.0,6.0,6.0,NaN,50,N,2.100000e+08,202104,5
6,42000000.0,NaN,NaN,NaN,NaN,NaN,NaN,1.0,4.0,1.0,4.0,9300000.0,51000000.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,40,N,2.600000e+08,202009,6
7,52000000.0,NaN,NaN,210000.0,200000.0,NaN,1.0,8.0,4.0,6.0,4.0,54000000.0,100000000.0,NaN,NaN,NaN,NaN,7.0,7.0,NaN,30,N,7.600000e+07,202208,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803273,3400000.0,59000000.0,94000000.0,3000000.0,3900000.0,66.0,11.0,158.0,91.0,17.0,25.0,6100000.0,9900000.0,NaN,NaN,NaN,1500000.0,9.0,NaN,9.0,50,N,5.700000e+07,202206,803273
803287,120000000.0,NaN,NaN,150000000.0,160000000.0,NaN,19.0,29.0,10.0,9.0,7.0,59000000.0,36000000.0,NaN,NaN,25000000.0,53000000.0,7.0,7.0,NaN,40,Y,1.000000e+09,202106,803287
803299,13000000.0,100000.0,100000.0,67000000.0,110000000.0,1.0,3.0,2.0,1.0,1.0,1.0,1800000.0,6200000.0,NaN,NaN,3500000.0,NaN,10.0,10.0,NaN,40,N,2.800000e+08,202209,803299
803305,NaN,64000000.0,95000000.0,NaN,NaN,3.0,NaN,11.0,3.0,1.0,2.0,19000000.0,4100000.0,NaN,NaN,9300000.0,4800000.0,5.0,5.0,NaN,40,Y,1.100000e+09,202104,803305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803308 entries, 0 to 803307
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   특정기간동안의 주식 매수체결횟수  268854 non-null  float64
 1   특정기간동안의 주식 매도체결횟수  257840 non-null  float64
 2   특정기간동안의 주식 매수종목수   268854 non-null  float64
 3   특정기간동안의 주식 매도종목수   257840 non-null  float64
 4   특정기간동안의 주식 매수액 합계  268854 non-null  float64
 5   특정기간동안의 주식 매도액 합계  257840 non-null  float64
dtypes: float64(6)
memory usage: 36.8 MB


,월말일 기준 예수금 등 현금성 자산액,월말일 기준 국내주식 자산 평가액,월말일 기준 국내주식 자산 매입액,월말일 기준 해외주식 자산 평가액,월말일 기준 해외주식 자산 매입액,월말일 기준 국내주식 보유종목수,월말일 기준 해외주식 보유종목수,특정기간동안의 주식 매수체결횟수,특정기간동안의 주식 매도체결횟수,특정기간동안의 주식 매수종목수,특정기간동안의 주식 매도종목수,특정기간동안의 주식 매수액 합계,특정기간동안의 주식 매도액 합계,특정기간동안의 주식 입고액 합계,특정기간동안의 주식 출고액 합계,특정기간동안의 입금액 합계,특정기간동안의 출금액 합계,특정기간동안의 HTS 또는 MTS 접속일수,특정기간동안의 MTS 접속일수,특정기간동안의 HTS 접속일수,10세 기준 연령대,관리사원 존재여부,과거 최고자산액,과거 최고자산액 기록 연월,고객번호
1,7700000.0,70000000.0,75000000.0,NaN,NaN,8.0,NaN,1.0,5.0,1.0,5.0,2200000.0,10000000.0,NaN,NaN,NaN,NaN,8.0,8.0,1.0,60,N,2.200000e+08,201711,1
2,1200.0,30000000.0,50000000.0,7300000.0,11000000.0,25.0,8.0,46.0,38.0,33.0,31.0,28000000.0,28000000.0,NaN,NaN,NaN,NaN,10.0,10.0,NaN,40,N,4.400000e+07,202206,2
3,47000000.0,NaN,NaN,NaN,NaN,NaN,NaN,6.0,8.0,2.0,2.0,300000000.0,300000000.0,NaN,NaN,NaN,NaN,1.0,1.0,NaN,40,Y,2.100000e+08,202201,3
5,1000000.0,86000000.0,150000000.0,NaN,NaN,2.0,NaN,9.0,9.0,3.0,4.0,11000000.0,15000000.0,NaN,NaN,4000000.0,4200000.0,8.0,8.0,NaN,50,N,2.100000e+08,202104,5
6,19000.0,44000000.0,45000000.0,NaN,NaN,3.0,NaN,14.0,9.0,5.0,4.0,310000000.0,290000000.0,NaN,NaN,NaN,NaN,10.0,10.0,NaN,40,N,2.600000e+08,202009,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803286,450000.0,130000000.0,120000000.0,NaN,NaN,1.0,NaN,20.0,14.0,1.0,1.0,87000000.0,70000000.0,NaN,NaN,450000.0,NaN,6.0,6.0,NaN,40,N,6.300000e+07,202204,803286
803287,100000000.0,NaN,NaN,190000000.0,200000000.0,NaN,20.0,39.0,7.0,13.0,4.0,29000000.0,8500000.0,NaN,NaN,98000000.0,67000000.0,8.0,8.0,NaN,40,Y,1.000000e+09,202106,803287
803290,120000.0,310000.0,390000.0,NaN,NaN,4.0,NaN,1.0,1.0,1.0,1.0,19000.0,23000.0,NaN,NaN,NaN,NaN,8.0,8.0,NaN,20,N,7.600000e+05,202205,803290
803304,400000.0,180000000.0,240000000.0,NaN,NaN,33.0,NaN,20.0,245.0,3.0,3.0,20000000.0,28000000.0,NaN,NaN,NaN,5300000.0,8.0,8.0,NaN,50,N,1.600000e+08,202204,803304


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 803308 entries, 0 to 803307
Data columns (total 6 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   특정기간동안의 주식 매수체결횟수  262016 non-null  float64
 1   특정기간동안의 주식 매도체결횟수  200271 non-null  float64
 2   특정기간동안의 주식 매수종목수   262016 non-null  float64
 3   특정기간동안의 주식 매도종목수   200271 non-null  float64
 4   특정기간동안의 주식 매수액 합계  262016 non-null  float64
 5   특정기간동안의 주식 매도액 합계  200271 non-null  float64
dtypes: float64(6)
memory usage: 36.8 MB


,월말일 기준 예수금 등 현금성 자산액,월말일 기준 국내주식 자산 평가액,월말일 기준 국내주식 자산 매입액,월말일 기준 해외주식 자산 평가액,월말일 기준 해외주식 자산 매입액,월말일 기준 국내주식 보유종목수,월말일 기준 해외주식 보유종목수,특정기간동안의 주식 매수체결횟수,특정기간동안의 주식 매도체결횟수,특정기간동안의 주식 매수종목수,특정기간동안의 주식 매도종목수,특정기간동안의 주식 매수액 합계,특정기간동안의 주식 매도액 합계,특정기간동안의 주식 입고액 합계,특정기간동안의 주식 출고액 합계,특정기간동안의 입금액 합계,특정기간동안의 출금액 합계,특정기간동안의 HTS 또는 MTS 접속일수,특정기간동안의 MTS 접속일수,특정기간동안의 HTS 접속일수,10세 기준 연령대,관리사원 존재여부,과거 최고자산액,과거 최고자산액 기록 연월,고객번호
3,51000000.0,NaN,NaN,NaN,NaN,NaN,NaN,23.0,16.0,2.0,2.0,620000000.0,610000000.0,NaN,NaN,7600000.0,NaN,3.0,NaN,3.0,40,Y,2.100000e+08,202201,3
5,53000000.0,83000000.0,150000000.0,NaN,NaN,2.0,NaN,69.0,28.0,3.0,3.0,230000000.0,190000000.0,NaN,NaN,35000000.0,3000000.0,9.0,9.0,NaN,50,N,2.100000e+08,202104,5
7,10000000.0,NaN,NaN,31000000.0,27000000.0,NaN,3.0,23.0,12.0,7.0,12.0,8400000.0,11000000.0,NaN,NaN,2900000.0,2500000.0,10.0,10.0,NaN,30,N,7.600000e+07,202208,7
14,3800000.0,93000000.0,84000000.0,NaN,NaN,22.0,NaN,52.0,3.0,5.0,3.0,11000000.0,470000.0,NaN,NaN,27000000.0,12000000.0,7.0,7.0,NaN,60,N,1.000000e+08,202104,14
15,100000.0,89000000.0,81000000.0,NaN,NaN,2.0,NaN,3.0,1.0,1.0,1.0,33000000.0,33000000.0,NaN,NaN,NaN,NaN,6.0,NaN,6.0,60,N,1.900000e+08,201507,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
803278,7700.0,23000000.0,17000000.0,NaN,NaN,4.0,NaN,1.0,1.0,1.0,1.0,1500000.0,1600000.0,NaN,NaN,NaN,NaN,8.0,8.0,NaN,20,N,1.300000e+07,202211,803278
803286,1800000.0,88000000.0,84000000.0,NaN,NaN,1.0,NaN,44.0,43.0,1.0,1.0,34000000.0,82000000.0,NaN,NaN,NaN,6000000.0,6.0,6.0,NaN,40,N,6.300000e+07,202204,803286
803287,130000000.0,NaN,NaN,200000000.0,190000000.0,NaN,16.0,15.0,5.0,8.0,5.0,16000000.0,12000000.0,NaN,NaN,16000000.0,6000000.0,7.0,7.0,NaN,40,Y,1.000000e+09,202106,803287
803295,1700000.0,59000000.0,120000000.0,3000000.0,4000000.0,18.0,14.0,3.0,6.0,3.0,6.0,1100000.0,1200000.0,NaN,NaN,NaN,NaN,6.0,6.0,NaN,40,N,2.000000e+08,202011,803295
